## estudiante

In [1]:
import random
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

class AgenteEstudiante:
    def __init__(self, grafo_conocimiento):
        self.conocimiento = grafo_conocimiento
        self.historial_errores = []
        self._inicializar_sistema_difuso()

    def _inicializar_sistema_difuso(self):
        self.dominio = ctrl.Antecedent(np.arange(0, 1.05, 0.05), 'dominio')
        self.estado = ctrl.Consequent(np.arange(0, 1.05, 0.05), 'estado')

        self.dominio['bajo'] = fuzz.trimf(self.dominio.universe, [0.0, 0.0, 0.5])
        self.dominio['medio'] = fuzz.trimf(self.dominio.universe, [0.3, 0.5, 0.7])
        self.dominio['alto'] = fuzz.trimf(self.dominio.universe, [0.5, 1.0, 1.0])

        self.estado['deficiente'] = fuzz.trimf(self.estado.universe, [0.0, 0.0, 0.5])
        self.estado['aceptable'] = fuzz.trimf(self.estado.universe, [0.3, 0.5, 0.7])
        self.estado['dominante'] = fuzz.trimf(self.estado.universe, [0.5, 1.0, 1.0])

        reglas = [
            ctrl.Rule(self.dominio['bajo'], self.estado['deficiente']),
            ctrl.Rule(self.dominio['medio'], self.estado['aceptable']),
            ctrl.Rule(self.dominio['alto'], self.estado['dominante'])
        ]

        self.sistema_ctrl = ctrl.ControlSystem(reglas)
        self.simulador = ctrl.ControlSystemSimulation(self.sistema_ctrl)

    def responder_pregunta(self, concepto):
        prob_acierto = self.conocimiento[concepto]
        acierta = random.random() < prob_acierto
        return acierta

    def generar_feedback(self, concepto, correcta):
        if correcta:
            return f"Bien hecho. Parece que comprendes {concepto}."
        else:
            ejemplo = self.obtener_ejemplo_error(concepto)
            return f"Fallaste en {concepto}. Revisa este ejemplo: {ejemplo}"

    def obtener_ejemplo_error(self, concepto):
        return f"Ejemplo relacionado con {concepto} que aclara el error."

    def actualizar_conocimiento(self, concepto, correcta):
        delta = 0.1 if correcta else 0.05
        self.conocimiento[concepto] = min(1.0, self.conocimiento[concepto] + delta)
        if not correcta:
            self.historial_errores.append((concepto, "respuesta incorrecta"))

    def evaluar_estado_cognitivo(self, nivel):
        self.simulador.input['dominio'] = nivel
        self.simulador.compute()
        return self.simulador.output['estado']

    def ciclo_aprendizaje(self, umbral=0.8, max_intentos=20):
        resultados = []
        for intento in range(max_intentos):
            for concepto in self.conocimiento:
                if self.conocimiento[concepto] < umbral:
                    correcta = self.responder_pregunta(concepto)
                    feedback = self.generar_feedback(concepto, correcta)
                    self.actualizar_conocimiento(concepto, correcta)
                    estado_cognitivo = self.evaluar_estado_cognitivo(self.conocimiento[concepto])
                    resultados.append({
                        "intento": intento + 1,
                        "concepto": concepto,
                        "correcta": correcta,
                        "nivel_post": self.conocimiento[concepto],
                        "estado_cognitivo": estado_cognitivo,
                        "feedback": feedback
                    })

            if all(nivel >= umbral for nivel in self.conocimiento.values()):
                break
        return pd.DataFrame(resultados)


Prueba 

In [3]:
def main():
    grafo_simulado = {
        "A*": 0.3,
        "Lógica Difusa": 0.2,
        "Redes Neuronales": 0.1
    }

    agente = AgenteEstudiante(grafo_simulado)
    df_resultados = agente.ciclo_aprendizaje()
    print("\nResultados del ciclo de aprendizaje:")
    print(df_resultados.to_string(index=False))


if __name__ == "__main__":
    main()


Resultados del ciclo de aprendizaje:
 intento         concepto  correcta  nivel_post  estado_cognitivo                                                                                                         feedback
       1               A*     False        0.35          0.322667                             Fallaste en A*. Revisa este ejemplo: Ejemplo relacionado con A* que aclara el error.
       1    Lógica Difusa     False        0.25          0.194444       Fallaste en Lógica Difusa. Revisa este ejemplo: Ejemplo relacionado con Lógica Difusa que aclara el error.
       1 Redes Neuronales     False        0.15          0.178205 Fallaste en Redes Neuronales. Revisa este ejemplo: Ejemplo relacionado con Redes Neuronales que aclara el error.
       2               A*      True        0.45          0.450189                                                                            Bien hecho. Parece que comprendes A*.
       2    Lógica Difusa     False        0.30          0.204167  